In [111]:
import pandas as pd
import seaborn as sns


In [ ]:
df = pd.read_csv("/data/6 class csv.csv")
df

,Temperature (K),Luminosity(L/Lo),Radius(R/Ro),Absolute magnitude(Mv),Star type,Star color,Spectral Class
0,3068,0.002400,0.1700,16.12,0,Red,M
1,3042,0.000500,0.1542,16.60,0,Red,M
2,2600,0.000300,0.1020,18.70,0,Red,M
3,2800,0.000200,0.1600,16.65,0,Red,M
4,1939,0.000138,0.1030,20.06,0,Red,M
...,...,...,...,...,...,...,...
235,38940,374830.000000,1356.0000,-9.93,5,Blue,O
236,30839,834042.000000,1194.0000,-10.63,5,Blue,O
237,8829,537493.000000,1423.0000,-10.73,5,White,A
238,9235,404940.000000,1112.0000,-11.23,5,White,A


In [113]:
df= df[df['Spectral Class']!='G']
df= df[df['Spectral Class']!='K']

In [114]:
color_map = {
    'Blue white': 'Blue-White',
    'blue-white': 'Blue-White',
    'Blue white ': 'Blue-White',
    'Blue-White': 'Blue-White',
    'Blue-white' : 'Blue-White',
    'Blue White': 'Blue-White',
    
    'White': 'White',
    'white': 'White',
    'Whitish': 'White',
    
    'Yellowish White': 'Yellow-White',
    'yellow-white': 'Yellow-White',
    'White-Yellow': 'Yellow-White',
    
    'Blue': 'Blue',
    'Blue ': 'Blue',
    
    'Red': 'Red',
    
    'Orange': 'Orange',
    'Orange-Red': 'Orange',
    
    'yellowish': 'Yellow',
    'Yellowish': 'Yellow',
    
    'Pale yellow orange': 'Yellow-Orange'
}


df['Star color'] = df['Star color'].replace(color_map)

In [115]:
X=df.drop("Spectral Class",axis=1)
y=df["Spectral Class"]

In [116]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(X,y,test_size=0.33,random_state=42)

In [117]:
from sklearn.preprocessing import OneHotEncoder
encoder = OneHotEncoder(handle_unknown='ignore', sparse_output=False)
encoder.fit(X_train)

X_train = encoder.transform(X_train)
X_test = encoder.transform(X_test)

In [118]:
from imblearn.over_sampling import SMOTE
from imblearn.pipeline import Pipeline
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import GridSearchCV, StratifiedKFold
from sklearn.metrics import accuracy_score
from sklearn.metrics import balanced_accuracy_score

# Step 1: Define SMOTE
smote = SMOTE(random_state=42)

# Step 2: Define the classifier
rf = RandomForestClassifier(random_state=42)

# Step 3: Create pipeline (SMOTE -> Classifier)
pipeline = Pipeline([
    ('smote', SMOTE(k_neighbors=4, random_state=42)),
    ('classifier', RandomForestClassifier(random_state=42))
])

# Step 4: Hyperparameter grid
param_grid = {
    'classifier__n_estimators': [50, 100],
    'classifier__max_depth': [None, 10],
    'classifier__min_samples_split': [2, 5],
    'classifier__min_samples_leaf': [1, 2],
    'smote__k_neighbors': [1]
}

# Step 5: Stratified K-Fold to preserve class ratios
cv = StratifiedKFold(n_splits=3, shuffle=True, random_state=42)

# Step 6: GridSearchCV with pipeline
grid_search = GridSearchCV(
    estimator=pipeline,
    param_grid=param_grid,
    scoring='accuracy',
    cv=cv,
    verbose=2,
    n_jobs=-1
)

# Step 7: Fit on training set
grid_search.fit(X_train, y_train)

# Step 8: Predict on test set
y_pred = grid_search.best_estimator_.predict(X_test)

# Step 9: Evaluate
print("Best Hyperparameters:", grid_search.best_params_)
print("Test Accuracy:", accuracy_score(y_test, y_pred))
print("Balanced Accuracy:", balanced_accuracy_score(y_test, y_pred))



Fitting 3 folds for each of 16 candidates, totalling 48 fits
Best Hyperparameters: {'classifier__max_depth': 10, 'classifier__min_samples_leaf': 1, 'classifier__min_samples_split': 2, 'classifier__n_estimators': 100, 'smote__k_neighbors': 1}
Test Accuracy: 0.935064935064935
Balanced Accuracy: 0.788888888888889


In [ ]:
print(y_train.value_counts())

Spectral Class
M    74
B    28
O    26
A    15
F    13
Name: count, dtype: int64


In [127]:
from collections import Counter
counts = Counter(df['Spectral Class'])
counts, len(df)

(Counter({'M': 111, 'B': 46, 'O': 40, 'A': 19, 'F': 17}), 233)

In [125]:
for k, v in counts.items():
    print(f'{k}: {v/len(df)}')

M: 0.47639484978540775
B: 0.19742489270386265
A: 0.0815450643776824
F: 0.07296137339055794
O: 0.17167381974248927
